In [1]:
import openpyxl
import glob
import os
from openpyxl import Workbook
import numpy as np
import pandas as pd
import xlwings as xw

In [2]:
# まとめた後のexcelファイル名
after_name = "gathering data"

# まとめる前のexcelファイルが保存されているパス
folderpath_before = "/Users/eiyou/Desktop/Python/data"
# まとめた後のexcelファイルを保存するパス
folderpath_after = "/Users/eiyou/Desktop/Python/summary"


# データを抽出するシート名
sheet_name = "Sheet"

# データを抽出する初めのセル
# Lのタイトル、値
title = 'B12'
val_star = 'C18'
val_col = 'C'

# Rのタイトル、値
tiR = 'C12'
valR = 'E18'
valR_col = 'E'



In [3]:
wb = Workbook()

files = sorted(glob.glob(folderpath_before + "/*.xlsx"))
new_ws = wb.active


# 整理後データを入れる空リストを用意
new_ws_titles = []
new_ws_values = []

# セルの初期座標を設定
x_one, y_one, y_two = 1, 1, 1


flg=True
# Lをまとめる
for file in files:

    opened_wb = openpyxl.load_workbook(file)
    sheet = opened_wb.worksheets[0]
    new_ws_title = sheet[title].value
    new_ws_titles.append(new_ws_title)
    new_ws.cell(x_one, y_one).value =  new_ws_title
    y_one += 1
    x_two = 2
    val_end = sheet.max_row
    for cel in sheet[val_star+':'+val_col +str(val_end)]:
        new_ws_value = cel[0].value#あたいの取得
        new_ws_values.append(new_ws_value)#リストに追加
        new_ws.cell(x_two, y_two).value =  new_ws_value
        x_two += 1
    y_two += 1
    
# Rをまとめる
for file in files:
    opened_wb = openpyxl.load_workbook(file)
    sheet = opened_wb.worksheets[0]
    new_ws_title = sheet[tiR].value
    new_ws_titles.append(new_ws_title)
    new_ws.cell(x_one, y_one).value =  new_ws_title
    y_one += 1
    x_two = 2
    val_end = sheet.max_row
    for cel in sheet[valR+':'+valR_col +str(val_end)]:
        new_ws_value = cel[0].value#あたいの取得
        new_ws_values.append(new_ws_value)#リストに追加
        new_ws.cell(x_two, y_two).value =  new_ws_value
        x_two += 1
    y_two += 1
    

# データが抽出できているか確認    
print(str(new_ws_titles))

wb.save(folderpath_after + "/" + after_name + ".xlsx")
wb.close()



print("次の処理を行ってください")

['corn oil 2%', 'corn oil 2%', 'corn oil 2%', 'corn oil 2%', 'sucrose 2%', 'sucrose 2%', 'sucrose 2%', 'sucrose 2%']
次の処理を行ってください


In [4]:
before_name = after_name


# データ解析のexcelファイル名
after_name = "Summary"

fileload_path = "/Users/eiyou/Desktop/Python/summary/"+before_name+".xlsx"
filesave_path = r"/Users/eiyou/Desktop/Python/summary"



date_flag=True#時刻データ 年/月/日 00:00:00 がありますか？ないならFalseに
first_time_cell_x, cell_y=18,2#その時刻データはどのセルにありますか？　セル座標をｘ，ｙの順で入力

data_interval=1#集計時間は何秒ですか？
Pause_criterion = 1#Pause_criterionは何秒ですか？

adjusted_criterion=int(1/data_interval*Pause_criterion)
#1/data_interval*Pause_criterion:使用上、データの集計間隔は1secを想定している。
#そのため、定数1を集計間隔data_intervalで割り、その倍率でPause_criterionを補正する。

In [5]:

wb2 = Workbook()
wb2 = openpyxl.load_workbook(fileload_path)
gathering_ws = wb2.worksheets[0]
excel = Workbook()
summary_ws = excel.active
excel.create_sheet(title='TimeAndIBI')
new_ws_time=excel.worksheets[-1]




def get_burst(arr, N):
    """最初に現れた0でない値から、その次の0がN連続するまでの間を取得
    """
    arr = np.r_[arr, np.zeros(N, dtype=int)]#最後にゼロを追加：ゼロがn回続けば一つの塊として処理するため、続かない場合のエラーを回避する
    is_nonzero = arr != 0# 0=F, 0>Tとすることで0>の区別を消す
    s = is_nonzero.argmax()# 始点＝最初に現れた0以外の値の位置(リスト内が論理演算のため、TRUEが大きく一番先に検出したTを返す)
    ln = np.convolve(is_nonzero[s:], np.ones(N, dtype=int), 'valid').argmin()# sより後ろで初めて0がN連続する箇所のindex
    #備考：論理演算子に対してconvolve関数（第２引数が１）でフィルタするとTを１に、Fを０にすることができる

    
    return arr[s:s+ln], (s, s+ln), s, ln#クラスターを返す、クラスターの開始時間マス、クラスターの時間, interBintervl

# 列の作成
abc = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
       'AA','AB','AC','AD','AE','AF','AG','AH','AI','AJ',
       'AK','AL','AM','AN','AO','AP','AQ','AR','AS','AT','AU','AV','AW','AX','AY','AZ','BA','BB','BC','BD','BE','BF','BG','BH','BI','BJ',
       'BK','BL','BM','BN','BO','BP','BQ','BR','BS','BT','BU','BV','BW','BX','BY','BZ']
    

# y座標を作成
index_two, index_ws_time=1,1



#最大列の値を取得
col_end =gathering_ws.max_column

data,time_list,IBI_list=[],[],[]

"""クラスターを算出する
"""
file_R_index=0
for index, j in enumerate(abc[:col_end]):
    datas= [cell.value for cell in gathering_ws[j]]
    datas.pop(0)

    if index >= col_end/2:
        opened_wb = openpyxl.load_workbook(files[file_R_index])
        sheet = opened_wb.worksheets[0]
        file_R_index+=1
    else :
        opened_wb = openpyxl.load_workbook(files[index])
        sheet = opened_wb.worksheets[0]
    
    for dat in datas:
        if dat is None:
            continue
        data.append(dat)
    a = np.array(data)
    j = []
    start_lick=[]
    time_list.clear()
    IBI_list.clear()
    pre_burst_end=0
    
    for i in range(a.size):
        out, [s, e], st_l, time= get_burst(a, adjusted_criterion)
        
        
        j.append(out.sum())#リック内の０も含むがmeanでなくsumなので問題なし
        time_list.append(time)#indexが返されるが、終了時刻は使用上ずれるため、加算しない
        start_lick.append(st_l)#indexが返される
        a[:e] = 0
        
        
        if (pre_burst_end > 0) and (out.sum() != 1):
            post_burst_str= st_l+1
            IBI_list.append(post_burst_str-pre_burst_end)
        
        if a.sum() == 0:
            break    

        if out.sum() != 1:
            pre_burst_end= st_l+1+time  
        
    if 1 in j:#リックが１回しかないクラスターを削除する
        del_list=[]
        del_list.clear()
        index_num = [n for n, v in enumerate(j) if v == 1]
        for index, v in enumerate(index_num):
            j.pop(v-index)
            start_lick.pop(v-index)
            time_list.pop(v-index)
            
    for index, v in enumerate(start_lick):#クラスターの開始時間を書き込む
        if date_flag == True:
            date = str(sheet.cell(first_time_cell_x+ v,cell_y ).value)
            target = ':'
            idx = date.find(target)-2
            r = date[idx:] 
            r2 = r.replace(' ', '')
            summary_ws.cell(index+3, index_two).value = r2
            summary_ws.cell(index+3, index_two).number_format= "h:mm:ss"
        else :
             summary_ws.cell(index+3, index_two).value = v/int(1/data_interval)
            #start_lickはセルのインデックスが収納されているため、補正する
            
    index_two += 1
    for index, v in enumerate(j):#クラスターサイズを記入
        summary_ws.cell(index+3, index_two).value = v
    index_two += 1
    for index, v in enumerate(time_list):#なめ時間を書き込む
        new_ws_time.cell(index+3, index_ws_time).value = v
    index_ws_time += 1
    for index, v in enumerate(IBI_list):#IBIを書き込む
        new_ws_time.cell(index+3, index_ws_time).value = v
    index_ws_time += 1
    
    data.clear()
    

header_list,header_xy=[],[]

header_list.clear
header_xy.clear
for headers in gathering_ws.iter_rows(min_row=1, max_row=1):#headerの取得
    for index, header in enumerate(headers):
        summary_ws.cell(1,2+index*2).value=header.value
        header_list.append(header.value)

summary_ws.insert_rows(2,1)#headrを入力
new_ws_time.insert_rows(1,1)#のちに入れるAverageIFs関数が値と参照するセルがx軸表（列）が同じでなければいけないため
summary_ws.cell(2,1).value, summary_ws.cell(2,2).value="burst start time", "burst size"
new_ws_time.cell(2,1).value, new_ws_time.cell(2,2).value="licking time", "IBI"     





"""新しいシートにデータをまとめる
"""


excel.create_sheet(title='Summary')
new2_ws=excel.worksheets[-1]

#新しいワークシートのheadrの取得
new2_ws.cell(1,2).value="Ave.Burst size(Palatability)"
new2_ws.cell(1,len(set(header_list))+3).value="Number of Burst(Motivation)"    
new2_ws.cell(1,len(set(header_list))*2+4).value="Total Licks(times)"     
new2_ws.cell(1,len(set(header_list))*3+5).value="Licking time(sec/burst)"
new2_ws.cell(1,len(set(header_list))*4+6).value="IBI(sec)"
for index, header in enumerate(sorted(set(header_list), key=header_list.index)):
    new2_ws.cell(1,3+index).value=header
    new2_ws.cell(1,4+len(set(header_list))+index).value=header    
    new2_ws.cell(1,5+len(set(header_list))*2+index).value=header
    new2_ws.cell(1,6+len(set(header_list))*3+index).value=header
    new2_ws.cell(1,7+len(set(header_list))*4+index).value=header

#時刻とラベル,データ処理順を記載
new2_ws.cell(1,1).value="time slot"
if date_flag == True:
    new2_ws.cell(2,1).value="00:00:00"
    new2_ws.cell(3,1).value="23:59:59"
else:
    new2_ws.cell(2,1).value=0
    new2_ws.cell(3,1).value=86401
    
new2_ws.cell(5,1).value="Data processing order"
for index, v in enumerate(files):
    new2_ws.cell(6+index,1).value= os.path.basename(v)

excel.save(filesave_path + "/" + after_name + ".xlsx")




'''
各個体のAVE、COUNT、SUMを算出する
'''
max_row = summary_ws.max_row
max_col = summary_ws.max_column
for cells in summary_ws.iter_rows(min_row=1, max_row=1):
    for cell in cells:
        if cell.value is not None:
            header_xy.append(cell)

x_axis =1
y_axis=0
header_num=len(set(header_list))





#関数を入力
for index, x in enumerate(header_xy):
        clock=str(summary_ws.cell(x.row+3, x.column-1).coordinate)+":"+str(summary_ws.cell(max_row, x.column-1).coordinate)#時刻    
        ref=str(summary_ws.cell(x.row+3, x.column).coordinate)+":"+str(summary_ws.cell(max_row, x.column).coordinate)#バーストサイズ     
        ref2=str(new_ws_time.cell(x.row+3, x.column-1).coordinate)+":"+str(new_ws_time.cell(max_row, x.column-1).coordinate)#
        ref3=str(new_ws_time.cell(x.row+3, x.column).coordinate)+":"+str(new_ws_time.cell(max_row, x.column).coordinate)#IBI
        
        if len(header_list) > 0:  
            header_list.pop(0)
        if date_flag == True:
            new2_ws.cell(1+x_axis,3+y_axis).value= '=IF(A3>A2,AVERAGEIFS(Sheet!'+str(ref)+',Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<="&A3),(SUMIFS(Sheet!'+str(ref)+',Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<=23:59:59")+SUMIFS(Sheet!'+str(ref)+',Sheet!'+str(clock)+',">=0:00",Sheet!'+str(clock)+',"<="&A3))/(COUNTIFS(Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<=23:59:59")+COUNTIFS(Sheet!'+str(clock)+',">=0:00",Sheet!'+str(clock)+',"<="&A3)))'
            new2_ws.cell(1+x_axis,4+header_num+y_axis).value= '=IF(A3>A2,COUNTIFS(Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<="&A3),COUNTIFS(Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<=23:59:59")+COUNTIFS(Sheet!'+str(clock)+',">=0:00",Sheet!'+str(clock)+',"<="&A3))'
            new2_ws.cell(1+x_axis,5+header_num*2+y_axis).value= '=IF(A3>A2,SUMIFS(Sheet!'+str(ref)+',Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<="&A3),SUMIFS(Sheet!'+str(ref)+',Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<=23:59:59")+SUMIFS(Sheet!'+str(ref)+',Sheet!'+str(clock)+',">=0:00",Sheet!'+str(clock)+',"<="&A3))'
            new2_ws.cell(1+x_axis,6+header_num*3+y_axis).value= '=IF(A3>A2,AVERAGEIFS(TimeAndIBI!'+str(ref2)+',Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<="&A3),(SUMIFS(TimeAndIBI!'+str(ref2)+',Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<=23:59:59")+SUMIFS(TimeAndIBI!'+str(ref2)+',Sheet!'+str(clock)+',">=0:00",Sheet!'+str(clock)+',"<="&A3))/(COUNTIFS(Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<=23:59:59")+COUNTIFS(Sheet!'+str(clock)+',">=0:00",Sheet!'+str(clock)+',"<="&A3)))'
            new2_ws.cell(1+x_axis,7+header_num*4+y_axis).value= '=IF(A3>A2,AVERAGEIFS(TimeAndIBI!'+str(ref3)+',Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<="&A3),(SUMIFS(TimeAndIBI!'+str(ref3)+',Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<=23:59:59")+SUMIFS(TimeAndIBI!'+str(ref3)+',Sheet!'+str(clock)+',">=0:00",Sheet!'+str(clock)+',"<="&A3))/(COUNTIFS(Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<=23:59:59")+COUNTIFS(Sheet!'+str(clock)+',">=0:00",Sheet!'+str(clock)+',"<="&A3)))'
        else :
            new2_ws.cell(1+x_axis,3+y_axis).value= '=AVERAGEIFS(Sheet!'+str(ref)+',Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<="&A3)'
            new2_ws.cell(1+x_axis,4+header_num+y_axis).value= '=COUNTIFS(Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<="&A3)'
            new2_ws.cell(1+x_axis,5+header_num*2+y_axis).value= '=SUMIFS(Sheet!'+str(ref)+',Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<="&A3)'
            new2_ws.cell(1+x_axis,6+header_num*3+y_axis).value= '=AVERAGEIFS(TimeAndIBI!'+str(ref2)+',Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<="&A3)'
            new2_ws.cell(1+x_axis,7+header_num*4+y_axis).value= '=AVERAGEIFS(TimeAndIBI!'+str(ref3)+',Sheet!'+str(clock)+',">="&A2,Sheet!'+str(clock)+',"<="&A3)'

        x_axis +=1

        if str(x.value) not in header_list:
            y_axis += 1
            x_axis =1
 
       
excel.save(filesave_path + "/" + after_name + ".xlsx")

#関数参照のエラーを回避するために更新
xb=xw.Book(filesave_path + "/" + after_name + ".xlsx")
xs=xb.sheets[0]
xs.range((1,1), (summary_ws.max_row,summary_ws.max_column)).value=xs.range((1,1), (summary_ws.max_row,summary_ws.max_column)).value

xb.save()
excel.close()
wb2.close()
wb.close()


print('終了しました')

終了しました


In [10]:
'''
ここのセルではShift+Enterを押さない
URLのlocalhostが8888でない場合、再起動する必要があるかも
'''


'\nここのセルではShift+Enterを押さない\nURLのlocalhostが8888でない場合、再起動する必要があるかも\n'